<a href="https://colab.research.google.com/github/samfreund/photonvision/blob/bump-rubik/OBB_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qai_hub torch ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 83.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.12.0.88
    Uninstalling opencv-python-4.12.0.88:
      Success

In [1]:
!wget https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n-obb.pt

--2025-10-29 23:07:32--  https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n-obb.pt
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/521807533/f75bd39c-f34b-499c-b3ec-e2df98c42f68?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-30T00%3A06%3A47Z&rscd=attachment%3B+filename%3Dyolo11n-obb.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-29T23%3A06%3A16Z&ske=2025-10-30T00%3A06%3A47Z&sks=b&skv=2018-11-09&sig=%2Fl0t3nMwM6rEiscTiZ09fL9aZxUBi09OajbYlO%2BLeFM%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MTc3OTU1MiwibmJmIjoxNzYxNzc5MjUyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ib

In [9]:
import qai_hub as hub
import torch
from ultralytics import YOLO

# hub.set_session_token("jpo8busq465wyuqmlumqpwee43dxz0it6zsgdw1d")

# Using pre-trained MobileNet
torch_model = YOLO("/content/yolo11n-obb.pt")
torch_model.model.eval()

# Step 1: Trace model
input_shape = (1, 3, 640, 640)
example_input = torch.rand(input_shape)
with torch.no_grad():
    traced_model = torch.jit.trace(torch_model.model, example_input, strict=False,
    check_inputs=[])

# Step 2: Compile model
compile_job = hub.submit_compile_job(
    model=traced_model,
    device=hub.Device("RB3 Gen 2 (Proxy)"),
    input_specs=dict(image=input_shape),
    options="--target_runtime tflite",
)

# Step 3: Download model
target_model = compile_job.get_target_model()
target_model.download("wheeeee.tflite")

AttributeError: 'NoneType' object has no attribute 'download'